In [8]:
import pandas as pd
import ast
import os
import json
import re
from itertools import islice
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
print(os.getpid())

987


In [2]:
file_path_synthetic = "/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/ultimate.jsonl"
file_path_real_train = "/mntssd/mnt3/shanshanbai/my_storage_from_qian/Data_Train_Test/first_paper/single_imbalance_train.csv"
file_path_real_test = "/mntssd/mnt3/shanshanbai/my_storage_from_qian/Data_Train_Test/first_paper/single_imbalance_test.csv"

In [ ]:
# concatenate train and test

# Load the two CSV files
csv1 = pd.read_csv(file_path_real_train)
csv2 = pd.read_csv(file_path_real_test)
# # Concatenate the files
# concatenated = pd.concat([csv1, csv2], ignore_index=True)
# # Save the result to a new CSV
# concatenated.to_csv('concatenated.csv', index=False)

In [ ]:
# Concatenate the files
concatenated = pd.concat([csv1, csv2], ignore_index=True)
# Save the result to a new CSV
concatenated.to_csv('/mntssd/mnt3/shanshanbai/my_storage_from_qian/Data_Train_Test/first_paper/selfBLEU.csv', index=False)

In [ ]:
concatenated = concatenated.drop(columns=['distance', 'tweet_id','user_id','created_time','latitude','longtitude','device','space_type','country', 'gom','Unnamed: 0','filepath', 'new_filepath','preprocessed_tweet'], inplace=False)

In [83]:
real.to_csv('/mntssd/mnt3/shanshanbai/my_storage_from_qian/Data_Train_Test/first_paper/selfBLEU__.csv', index=False)

In [ ]:
# Regex pattern to match URLs
url_pattern = r'https?://\S+|www\.\S+'

# Remove URLs and create a new column
concatenated['tweet_no_url'] = concatenated['original_tweet'].apply(lambda x: re.sub(url_pattern, '', x))


In [101]:
path_real = "/mntssd/mnt3/shanshanbai/my_storage_from_qian/Data_Train_Test/first_paper/selfBLEU__.csv"
path_synthetic = "/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/ultimate.jsonl"
real = pd.read_csv(path_real)
synthetic = pd.read_json(path_synthetic, lines=True)


In [106]:
residential = synthetic[:6558]
commercial = synthetic[6558:]
residential['mapped_class'] = "residential"
commercial['mapped_class'] = "commercial"

In [109]:
# Concatenate the two DataFrames
synthetic = pd.concat([residential, commercial], ignore_index=True)

In [111]:
real.head(5)

In [110]:
synthetic.head(5)

In [81]:
# real = real.groupby('building_id').agg({
#     'mapped_class': 'first',
#     'language': lambda x: list(x),
#     'city': 'first',
#     'tweet_no_url': lambda x: list(x),
# }).reset_index()

In [82]:
real.head(5)

In [112]:
# Find overlapping building_id values
overlap = pd.merge(real, synthetic, on='building_id', how='inner')['building_id']
# Filter both DataFrames to only include overlapping building_id
real_filtered = real[real['building_id'].isin(overlap)]
synthetic_filtered = synthetic[synthetic['building_id'].isin(overlap)]

In [9]:
real_filtered = pd.read_csv('/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/real.csv')
synthetic_filtered = pd.read_csv('/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/synthetic.csv')

In [10]:
synthetic_filtered['tweets'] = synthetic_filtered['tweets'].apply(ast.literal_eval)

In [11]:
all_tweets = [tweet for sublist in synthetic_filtered['tweets'] for tweet in sublist]

In [12]:
len(all_tweets)

22444

In [13]:
# Function to calculate self-BLEU
def calculate_self_bleu(tweets):
    scores = []
    smoothie = SmoothingFunction().method4  # Optional smoothing for BLEU
    
    for i, candidate in enumerate(tweets):
        references = [tweet.split() for j, tweet in enumerate(tweets) if i != j]
        candidate_tokens = candidate.split()
        bleu_score = sentence_bleu(references, candidate_tokens, smoothing_function=smoothie)
        scores.append(bleu_score)
    
    return scores

In [ ]:
# Calculate self-BLEU for each tweet
synthetic_filtered['self_BLEU'] = calculate_self_bleu(all_tweets)

# Calculate the average self-BLEU for the dataset
average_self_bleu = synthetic_filtered['self_BLEU'].mean()
print(f"Average Self-BLEU: {average_self_bleu}")